# Data Classes and MongoDB

In this notebook, I will briefly describe how to set up some simple, astronomically-significant data classes and load them into a MongoDB NoSQL database for future use.

To run this notebook, you need to run `Python 3.7` or later and have `pymongo` installed.

### Data Classes

Data classes were introduced in Python 3.7 and offer an easy way to quickly create python classes for storing rich, structured data. Let's create a simple class to store brown dwarf objects.

In [1]:
from dataclasses import dataclass, field, asdict

@dataclass
class BrownDwarf_v0:
    name: str
    ra: float
    dec: float
    name_list: list = field(default_factory=list)

In [2]:
bd = BrownDwarf_v0(name='1207-3932', ra=181.889, dec=-39.548)
bd.name_list = ['TWA 27', '2MASS J12073346-3932539']

In [3]:
bd

BrownDwarf_v0(name='1207-3932', ra=181.889, dec=-39.548, name_list=['TWA 27', '2MASS J12073346-3932539'])

As an alternative, you can have a separate dataclass to store the ra/dec information:

In [4]:
@dataclass
class Coords:
    ra: float
    dec: float
        
@dataclass
class BrownDwarf_v1:
    name: str
    coords: Coords
    name_list: list = field(default_factory=list)

In [5]:
c = Coords(ra=181.889, dec=-39.548)
bd = BrownDwarf_v1(name='1207-3932', coords=c)
print(bd)

BrownDwarf_v1(name='1207-3932', coords=Coords(ra=181.889, dec=-39.548), name_list=[])


Or get even fancier and store an astropy SkyCoords object, if you wanted:

In [6]:
from astropy.coordinates import SkyCoord
import astropy.units as u

@dataclass
class BrownDwarf_v2:
    name: str
    coords: SkyCoord
    name_list: list = field(default_factory=list)

In [7]:
s = SkyCoord(ra=181.889*u.deg, dec=-39.548*u.deg)
bd = BrownDwarf_v2(name='1207-3932', coords=s)
print(bd)

BrownDwarf_v2(name='1207-3932', coords=<SkyCoord (ICRS): (ra, dec) in deg
    (181.889, -39.548)>, name_list=[])


As a SkyCoord object, you get all the usual functionality you expect. For example, you can quickly check the constellation:

In [8]:
print(bd.coords.get_constellation())

Centaurus


For the purposes of this demo, I'll stick to using custom-built objects to store my data.
Let's define a more complete example to work with:

In [9]:
@dataclass
class Coords:
    ra: float
    dec: float
        
@dataclass
class Photometry:
    value: float
    error: float
    unit: str = 'mag'
        
@dataclass
class BrownDwarf:
    source_id: int
    name: str
    coords: Coords
    J: Photometry = None
    H: Photometry = None
    Ks: Photometry = None
    spectral_type: str = None
    name_list: list = field(default_factory=list)

In [10]:
c = Coords(ra=181.889, dec=-39.548)
# 2MASS_J: 12.99 +/- 0.03
# 2MASS_H: 12.39 +/- 0.03
# 2MASS_Ks: 11.95 +/- 0.03
j = Photometry(12.99, 0.03)
bd = BrownDwarf(source_id=11, name='1207-3932', coords=c,
               J = Photometry(12.99, 0.03),
               H = Photometry(12.39, 0.03),
               Ks = Photometry(11.95, 0.03))
bd.name_list = ['TWA 27', '2MASS J12073346-3932539']
bd.spectral_type = 'M8.0'
print(bd)

BrownDwarf(source_id=11, name='1207-3932', coords=Coords(ra=181.889, dec=-39.548), J=Photometry(value=12.99, error=0.03, unit='mag'), H=Photometry(value=12.39, error=0.03, unit='mag'), Ks=Photometry(value=11.95, error=0.03, unit='mag'), spectral_type='M8.0', name_list=['TWA 27', '2MASS J12073346-3932539'])


Now that we have a representation of the basic parameters we want to store for our object, we can loop over some table or input the values we need to store. At some point, though, we'll want to save our work in some more concrete fashion. For that, we'll use JSON and the `asdict` function in dataclasses, which is similar to the built-in class method of `__dict__`:

In [11]:
asdict(bd)
#bd.__dict__

{'source_id': 11,
 'name': '1207-3932',
 'coords': {'ra': 181.889, 'dec': -39.548},
 'J': {'value': 12.99, 'error': 0.03, 'unit': 'mag'},
 'H': {'value': 12.39, 'error': 0.03, 'unit': 'mag'},
 'Ks': {'value': 11.95, 'error': 0.03, 'unit': 'mag'},
 'spectral_type': 'M8.0',
 'name_list': ['TWA 27', '2MASS J12073346-3932539']}

This is a dictionary representation of our object. To make it even more human-readable we can use the `json` package:

In [12]:
import json

print(json.dumps(asdict(bd), default=lambda x: x.__dict__, indent=4))

{
    "source_id": 11,
    "name": "1207-3932",
    "coords": {
        "ra": 181.889,
        "dec": -39.548
    },
    "J": {
        "value": 12.99,
        "error": 0.03,
        "unit": "mag"
    },
    "H": {
        "value": 12.39,
        "error": 0.03,
        "unit": "mag"
    },
    "Ks": {
        "value": 11.95,
        "error": 0.03,
        "unit": "mag"
    },
    "spectral_type": "M8.0",
    "name_list": [
        "TWA 27",
        "2MASS J12073346-3932539"
    ]
}


In the above example, I used `default= lambda x: x.__dict__` to tell json.dumps that by default it should attempt to use the dictionary representation of classes if it found cases it could not understand. This, or something similar, is needed to recursively convert any nested dataclasses you may have built (such as Photometry and Coords)

### MongoDB

You may be wondering why use JSON at all? Why not just flatten it out and write a long table? The reason is that I want to use MongoDB to store my data. MongoDB is a NoSQL database that relies on JSON to store its documents. In fact, it's explicitly a type of database known as a document-store. By representing my data as JSON, I have a format that I can directly store into MongoDB without any major work.

You can download a free copy of MongoDB from [https://www.mongodb.com/](https://www.mongodb.com/) and can run a server locally on your machine, which is what I've done. Alternatively you can connect to a Cloud instance, if you have access to one already or sign up for their Atlas service.

Let's connect to a local MongoDB server instance; you may need to start this instance separately, refer to the mongodb documentation. In my case, I had to run `mongod --dbpath PATH-TO-DB-DIR` in a Bash terminal, where `PATH-TO-DB-DIR` is the directory where I store my mongodb databases:

In [13]:
import pymongo

client = pymongo.MongoClient()  # default connection (ie, local)

Now, we can specify the database we'd like to use, as well as the *collection*. A MongoDB collection is the equivalent of a table in relational databases like SQL. Each collection is built up of multiple documents (equivalent to rows or entries). Unlike relational database, neither the database or collection is required to exist prior to loading documents into it. If one doesn't exist, it will be created when you load your first document. If you've been running this tutorial several times, you may already have a collection. If you want to clear it you can use the `.drop()` method on it.

In [14]:
db_name = 'test'
db = client[db_name]  # database
dwarfs = db.dwarfs  # collection; can also call as db['dwarfs']
dwarfs.drop()  # drop collection, if needed

Now, let's load up that JSON representation of the brown dwarf we saved (we need an actual JSON object, not the string representation we produced before):

In [15]:
json_data = json.loads(json.dumps(asdict(bd), default=lambda x: x.__dict__))
result = dwarfs.insert_one(json_data)

### Example Data Load

Let's load up a bunch of data first so we can better explore how to use mongodb. Here is a small sample of data from the BDNYC Brown Dwarf database. For simplicity, I've only included J and H 2MASS data and only a single spectral type estimate. For more details on the BDNYC database, I'll refer you to [http://database.bdnyc.org](http://database.bdnyc.org)

In [16]:
bddata = """#id	sname	ra	dec	sptype	J	J_err	H	H_err	names
2	1331-0116	202.95387	-1.280556	16	15.46	0.04	14.47	0.04	SDSS J133148.92-011651.4, 2MASS J13314894-0116500
4	1448+1031	222.106791	10.533056	13.5	14.556	0.034	13.433	0.033	2MASS J14482563+1031590, 2MASSW J1448256+103159, SDSS J144825.69+103158.8, 2MUCD 20587, ULAS2MASS J1448+1031, 2MASSW J1448256+103159
7	1439+1929	219.868167	19.487472	11	12.759	0.019	12.041	0.019	2MASS J14392837+1929150, 2MASSW J1439284+192915
14	2249+0044	342.472709	0.734611	11	16.587	0.125	15.421	0.109	SDSSp J224953.45+004404.2, 2M2249+0044
15	2208+2921	332.05679	29.355972	13	15.797	0.085	14.793	0.071	2MASSW J2208136+292121, 2MASS J22081363+2921215, 2MASSW J2208136+292121
17	0027+0503	6.924875	5.061583	8	16.189	0.093	15.288	0.099	2MASS J00274197+0503417, PC 0025+0447
19	2148+4003	327.068041	40.0665	16	14.147	0.029	12.783	0.03	2MASS J21481628+4003593, 2MASS J21481633+4003594, 2MASS J21481633+4003594, 2MASSW J2148162+400359, 2MASS J21481628+4003593
20	1102-3430	165.54097	-34.509869	8.5	13.034	0.024	12.356	0.022	TWA 28, 2MASS J11020983-3430355, SSSPM 1102-3431
32	0415-0935	63.831417	-9.585167	28	15.695	0.058	15.537	0.113	2MASSI J0415195-093506, 2MASS J04151954-0935066, WISEP J041521.20-093500.6
34	0727+1710	111.826001	17.167	27	15.6	0.061	15.756	0.171	2MASSI J0727182+171001, 2MASS J07271824+1710012, WISEP J072719.13+170952.0, LEHPM 2-461
36	0451-3402	72.753833	-34.0375	10.5	13.541	0.023	12.826	0.023	2MASSI J0451009-340214, V* SY Cae, 2MASS J04510093-3402150, 2MASSI J0451009-340214
53	1515+4847	228.753459	48.794889	16	14.111	0.029	13.099	0.031	2MASSW J1515008+484742, 2MASS J15150083+4847416, SDSS J151500.62+484744.8, 2MUCD 11314
61	1245-4429	191.309	-44.485477	9	14.518	0.032	13.8	0.033	TWA 29, 2MASS J12451416-4429077, DENIS J124514.1-442907
63	0334-4953	53.537667	-49.893944	9	11.376	0.023	10.823	0.026	2MASS J03341218-4953322, DENIS-P J033411.3-495333, 2MASSI J0334114-495334, LEHPM 3396
80	1552+2948	238.24591	29.81342	10	13.478	0.026	12.606	0.026	2MASSW J1552591+294849, 2MASS J15525906+2948485, 2MASSW J1552591+294849, SDSS J155259.01+294848.3
82	1835+3259	278.90792	32.998497	8.5	10.27	0.022	9.617	0.021	2MASS J18353790+3259545, LSR J1835+3259
83	1547-2423	236.94662	-24.397028	9	13.97	0.029	13.271	0.034	2MASS J15474719-2423493, DENIS J154747.2-242349
86	0036+1821	9.067376	18.352889	13.5	12.466	0.027	11.588	0.03	2MASSW J0036159+182110, 2MASS J00361617+1821104
91	0518-2756	79.692333	-27.946028	10	15.262	0.043	14.295	0.046	2MASS J05184616-2756457, 2MASS J05184616-2756457, 2MASSI J0518461-275645, 2MUCD 10381
96	0248-1651	42.170846	-16.856022	8	12.551	0.022	11.872	0.022	LP 771-21, 2MASS J02484096-1651249, BR 0246-1703, WISEP J024840.95-165124.8
98	0241-0326	40.297958	-3.449639	10	15.799	0.065	14.811	0.053	2MASS J02411151-0326587, 2MASS J02411151-0326587, 2MASSI J0241115-032658"""

In [17]:
data = []
for row in bddata.split('\n'):
    if row.startswith('#'): continue
    elems = row.split('\t')
    
    # Format the spectral type
    spnum = float(elems[4])
    if spnum >= 20:
        sptype = 'T{:3.1f}'.format(spnum-20)
    elif spnum >= 10:
        sptype = 'L{:3.1f}'.format(spnum-10)
    else:
        sptype = 'M{:3.1f}'.format(spnum)
        
    temp = BrownDwarf(source_id=int(elems[0]),
                      name=elems[1], 
                      coords=Coords(ra=float(elems[2]), dec=float(elems[3])),
                      spectral_type=sptype,
                      J=Photometry(value=float(elems[5]), error=float(elems[6])),
                      H=Photometry(value=float(elems[7]), error=float(elems[8])),
                      name_list=elems[9].split(',')
                     )
    data.append(temp)

Now, if we check the data variable we can see that it is *literally* a `list` of `BrownDwarf`s:

In [18]:
for i, row in enumerate(data):
    if i>4: break  # only display the first 5
    print(row)

BrownDwarf(source_id=2, name='1331-0116', coords=Coords(ra=202.95387, dec=-1.280556), J=Photometry(value=15.46, error=0.04, unit='mag'), H=Photometry(value=14.47, error=0.04, unit='mag'), Ks=None, spectral_type='L6.0', name_list=['SDSS J133148.92-011651.4', ' 2MASS J13314894-0116500'])
BrownDwarf(source_id=4, name='1448+1031', coords=Coords(ra=222.106791, dec=10.533056), J=Photometry(value=14.556, error=0.034, unit='mag'), H=Photometry(value=13.433, error=0.033, unit='mag'), Ks=None, spectral_type='L3.5', name_list=['2MASS J14482563+1031590', ' 2MASSW J1448256+103159', ' SDSS J144825.69+103158.8', ' 2MUCD 20587', ' ULAS2MASS J1448+1031', ' 2MASSW J1448256+103159'])
BrownDwarf(source_id=7, name='1439+1929', coords=Coords(ra=219.868167, dec=19.487472), J=Photometry(value=12.759, error=0.019, unit='mag'), H=Photometry(value=12.041, error=0.019, unit='mag'), Ks=None, spectral_type='L1.0', name_list=['2MASS J14392837+1929150', ' 2MASSW J1439284+192915'])
BrownDwarf(source_id=14, name='2249+

We can now loop over these and load them up into our database. One thing I do here is check the source_id value first to avoid re-inserting an existing brown dwarf (since the source_id's are unique in the core BDNYC database). It's an optional step I take to avoid duplicated documents.

In [19]:
for row in data:
    source_id = row.source_id
    json_data = json.loads(json.dumps(asdict(row), default=lambda x: x.__dict__))
    
    count = dwarfs.count_documents({'source_id': source_id})

    if count > 0:
        # Replace existing
        cursor = dwarfs.find({'source_id': source_id})
        for doc in cursor:
            result = dwarfs.replace_one({'_id': doc['_id']}, json_data)
    else:
        # Insert new
        result = dwarfs.insert_one(json_data)

### Database Queries

Now, we can examine the data in the database with standard MongoDB queries. Below are a few examples, but I encourage you to read through the [MongoDB](https://docs.mongodb.com/manual/tutorial/query-documents/) and [pymongo](http://api.mongodb.com/python/current/index.html) documentation for more details. Note that the `_id` field is automatically generated by MongoDB when storing the document.

In [20]:
count = dwarfs.count_documents({})
print('Total documents: ', count)
cursor = dwarfs.find({'spectral_type': 'M8.0'})
for doc in cursor:
    print(doc)

Total documents:  22
{'_id': ObjectId('5cd862122a37510c615e69c8'), 'source_id': 11, 'name': '1207-3932', 'coords': {'ra': 181.889, 'dec': -39.548}, 'J': {'value': 12.99, 'error': 0.03, 'unit': 'mag'}, 'H': {'value': 12.39, 'error': 0.03, 'unit': 'mag'}, 'Ks': {'value': 11.95, 'error': 0.03, 'unit': 'mag'}, 'spectral_type': 'M8.0', 'name_list': ['TWA 27', '2MASS J12073346-3932539']}
{'_id': ObjectId('5cd862122a37510c615e69ce'), 'source_id': 17, 'name': '0027+0503', 'coords': {'ra': 6.924875, 'dec': 5.061583}, 'J': {'value': 16.189, 'error': 0.093, 'unit': 'mag'}, 'H': {'value': 15.288, 'error': 0.099, 'unit': 'mag'}, 'Ks': None, 'spectral_type': 'M8.0', 'name_list': ['2MASS J00274197+0503417', ' PC 0025+0447']}
{'_id': ObjectId('5cd862122a37510c615e69dc'), 'source_id': 96, 'name': '0248-1651', 'coords': {'ra': 42.170846, 'dec': -16.856022}, 'J': {'value': 12.551, 'error': 0.022, 'unit': 'mag'}, 'H': {'value': 11.872, 'error': 0.022, 'unit': 'mag'}, 'Ks': None, 'spectral_type': 'M8.0', '

We can 'project', or return only the fields we are interested in by suppling a second parameter to our queries:

In [21]:
cursor = dwarfs.find({'spectral_type': 'M8.0'}, {'_id':0, 'source_id': 1, 'name': 1, 'spectral_type': 1})
for doc in cursor:
    print(doc)

{'source_id': 11, 'name': '1207-3932', 'spectral_type': 'M8.0'}
{'source_id': 17, 'name': '0027+0503', 'spectral_type': 'M8.0'}
{'source_id': 96, 'name': '0248-1651', 'spectral_type': 'M8.0'}


With a large dataset, you can create indices to better search for your data. The index will update as more data is added to it. Or you can always drop and re-create it. Here's an example of creating a text index on the name and name_list fields:

In [22]:
#dwarfs.drop_index('text_fields')
dwarfs.create_index([('name', pymongo.TEXT),
                     ('name_list', pymongo.TEXT)],
                      name='text_fields',
                      background=True)

'text_fields'

In [23]:
cursor = dwarfs.find({'$text': {'$search': 'TWA'}}, {'_id': 0, 'name': 1, 'name_list': 1})
for doc in cursor:
    print(doc)

{'name': '1245-4429', 'name_list': ['TWA 29', ' 2MASS J12451416-4429077', ' DENIS J124514.1-442907']}
{'name': '1102-3430', 'name_list': ['TWA 28', ' 2MASS J11020983-3430355', ' SSSPM 1102-3431']}
{'name': '1207-3932', 'name_list': ['TWA 27', '2MASS J12073346-3932539']}


For partial name matching, you can also use regular expressions:

In [24]:
cursor = dwarfs.find({'name_list': {'$regex': '12'}}, {'_id': 0, 'name': 1, 'name_list': 1})
for doc in cursor:
    print(doc)

{'name': '1207-3932', 'name_list': ['TWA 27', '2MASS J12073346-3932539']}
{'name': '2208+2921', 'name_list': ['2MASSW J2208136+292121', ' 2MASS J22081363+2921215', ' 2MASSW J2208136+292121']}
{'name': '0727+1710', 'name_list': ['2MASSI J0727182+171001', ' 2MASS J07271824+1710012', ' WISEP J072719.13+170952.0', ' LEHPM 2-461']}
{'name': '1245-4429', 'name_list': ['TWA 29', ' 2MASS J12451416-4429077', ' DENIS J124514.1-442907']}
{'name': '0334-4953', 'name_list': ['2MASS J03341218-4953322', ' DENIS-P J033411.3-495333', ' 2MASSI J0334114-495334', ' LEHPM 3396']}
{'name': '0248-1651', 'name_list': ['LP 771-21', ' 2MASS J02484096-1651249', ' BR 0246-1703', ' WISEP J024840.95-165124.8']}


If you wanted to, you could now go back and re-create the Python `BrownDwarf` class objects with the data from the database. In practice, this is a little tricky since the JSON return doesn't explictly tell you what dataclass it came from. Below is a rough example of how you could manually re-build the dataclass, though I've seen StackOverflow examples that can set this up a bit more automatically.

In [25]:
cursor = dwarfs.find({'source_id': 11})
doc = list(cursor)[0]
del doc['_id']
print(doc)

{'source_id': 11, 'name': '1207-3932', 'coords': {'ra': 181.889, 'dec': -39.548}, 'J': {'value': 12.99, 'error': 0.03, 'unit': 'mag'}, 'H': {'value': 12.39, 'error': 0.03, 'unit': 'mag'}, 'Ks': {'value': 11.95, 'error': 0.03, 'unit': 'mag'}, 'spectral_type': 'M8.0', 'name_list': ['TWA 27', '2MASS J12073346-3932539']}


In [26]:
j = Photometry(**doc['J'])
h = Photometry(**doc['H'])
ks = Photometry(**doc['Ks'])
c = Coords(**doc['coords'])
print(j)
print(c)

Photometry(value=12.99, error=0.03, unit='mag')
Coords(ra=181.889, dec=-39.548)


In [27]:
del doc['coords']
del doc['J']
del doc['H']
del doc['Ks']
new_bd = BrownDwarf(**doc, coords=c, J=j, H=h, Ks=ks)
print(new_bd)

BrownDwarf(source_id=11, name='1207-3932', coords=Coords(ra=181.889, dec=-39.548), J=Photometry(value=12.99, error=0.03, unit='mag'), H=Photometry(value=12.39, error=0.03, unit='mag'), Ks=Photometry(value=11.95, error=0.03, unit='mag'), spectral_type='M8.0', name_list=['TWA 27', '2MASS J12073346-3932539'])


In [28]:
# Original, for comparison
print(bd)

BrownDwarf(source_id=11, name='1207-3932', coords=Coords(ra=181.889, dec=-39.548), J=Photometry(value=12.99, error=0.03, unit='mag'), H=Photometry(value=12.39, error=0.03, unit='mag'), Ks=Photometry(value=11.95, error=0.03, unit='mag'), spectral_type='M8.0', name_list=['TWA 27', '2MASS J12073346-3932539'])


In [29]:
bd == new_bd

True

There are a lot more ways you can query this database and, as you can image, lots more ways to create and work with dataclasses. I'll leave it up to the reader to examine the documentation and play around with the code. I can recommend downloading the MongoDB Compass application from [https://www.mongodb.com/products/compass](https://www.mongodb.com/products/compass) which provides a nice GUI to directly access your database. Some advanced queries aren't possible in it, but it can serve as useful introduction to how to explore the data.

![compass screenshot](../figures/mongodb_compass.png "Example Compass Screenshot")